In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json

def pretty(d):
    print(json.dumps(d, indent=2, sort_keys=True, default=lambda o: str(o)))

In [3]:
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '/home/gabriel/credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

# Retrieving schedule calendars

The filter by the prefix is just for illustration. Each created schedule would be stored in the database pointing it to the worker it related to.

In [4]:
all_calendars = service.calendarList().list().execute()
schedule_name_prefix = "[RDC]"

for calendar in all_calendars['items']:
    if calendar['summary'].startswith(schedule_name_prefix):
        print(f"{calendar['summary']}: {calendar['id']}")

worker_schedules = [
    {'id': calendar['id']}
    for calendar in all_calendars['items']
    if calendar['summary'].startswith(schedule_name_prefix)
]

[RDC] Agenda do Washington: h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com
[RDC] Agenda do Robson: 8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com


# Creating a schedule for a worker

In [5]:
calendars_service = service.calendars()
timezone = 'America/Sao_Paulo'
body = {
    "kind": "calendar#calendar",
    "summary": f"{schedule_name_prefix} Agenda do Robson",
    "timeZone": timezone
}
request = calendars_service.insert(body=body)
# request.execute()

# Getting free time slots from workers

In [21]:
import arrow, datetime

timezone = 'America/Sao_Paulo'
working_day_start = arrow.get(datetime.datetime(2020, 3, 19, 18), timezone)
working_day_end = arrow.get(datetime.datetime(2020, 3, 19, 21, 30), timezone)

freebusy_response = service.freebusy().query(body={
    "calendarExpansionMax": len(worker_schedules),
    "groupExpansionMax": 0,
    "timeMax": working_day_end.isoformat(),
    "items": worker_schedules,
    "timeMin": working_day_start.isoformat(),
    "timeZone": timezone
}).execute()

def busy_item_to_dates(busy_item):
    return (arrow.get(busy_item['start']), arrow.get(busy_item['end']))

busy_times_by_calendar = {
    calendar_id: [busy_item_to_dates(busy_item) for busy_item in calendar['busy']]
    for calendar_id, calendar in freebusy_response['calendars'].items()
}

In [22]:
pretty(freebusy_response)

{
  "calendars": {
    "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com": {
      "busy": []
    },
    "h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com": {
      "busy": [
        {
          "end": "2020-03-19T20:30:00-03:00",
          "start": "2020-03-19T19:30:00-03:00"
        }
      ]
    }
  },
  "kind": "calendar#freeBusy",
  "timeMax": "2020-03-20T00:30:00.000Z",
  "timeMin": "2020-03-19T21:00:00.000Z"
}


In [23]:
pretty(busy_times_by_calendar)

{
  "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com": [],
  "h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com": [
    [
      "2020-03-19T19:30:00-03:00",
      "2020-03-19T20:30:00-03:00"
    ]
  ]
}


In [62]:
import random
import logging


def get_timeslots_to_analyse(start_time, end_time):
    time_being_analysed = start_time
    while time_being_analysed < end_time:
        yield (time_being_analysed, time_being_analysed.shift(minutes=15))
        time_being_analysed = time_being_analysed.shift(minutes=15)

        
def is_timeslot_outside_range(timeslot, date_range):
    comes_before_range = (timeslot[0] <= date_range[0] and timeslot[1] <= date_range[0])
    comes_after_range = (timeslot[0] >= date_range[1] and timeslot[1] >= date_range[1])
    
    return comes_before_range or comes_after_range


def is_calendar_available_for_timeslot(busy_times, timeslot):
    if not busy_times:
        return True
    
    timeslot_not_conflicting = all(
        is_timeslot_outside_range(timeslot, busy_time)
        for busy_time in busy_times
    )
    
    return timeslot_not_conflicting


def is_timeslot_available(timeslot, busy_times_by_calendar):
    calendars = [*busy_times_by_calendar.keys()]
    random.shuffle(calendars)
    
    for calendar in calendars:
        busy_times = busy_times_by_calendar[calendar]
        
        if is_calendar_available_for_timeslot(busy_times, timeslot):
            logging.debug(f"{timeslot[0].format('HH:mm')}-{timeslot[1].format('HH:mm')}: {calendar}")
            return True, calendar

    return False, None


def get_one_free_timeslot_by_hour(working_day_start, working_day_end):
    timeslot_by_hour = {}

    for timeslot in get_timeslots_to_analyse(working_day_start, working_day_end):
        if timeslot[0].hour in timeslot_by_hour:
            continue
        
        available, calendar = is_timeslot_available(timeslot, busy_times_by_calendar)
        
        if available:
            timeslot_by_hour[timeslot[0].hour] = {
                'timeslot': timeslot,
                'calendar': calendar
            }

    return [*timeslot_by_hour.values()]

In [65]:
free_timeslots = get_one_free_timeslot_by_hour(working_day_start, working_day_end)
pretty(free_timeslots)

[
  {
    "calendar": "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com",
    "timeslot": [
      "2020-03-19T18:00:00-03:00",
      "2020-03-19T18:15:00-03:00"
    ]
  },
  {
    "calendar": "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com",
    "timeslot": [
      "2020-03-19T19:00:00-03:00",
      "2020-03-19T19:15:00-03:00"
    ]
  },
  {
    "calendar": "8ne3s7igp7vvikg00cm3ml9c3c@group.calendar.google.com",
    "timeslot": [
      "2020-03-19T20:00:00-03:00",
      "2020-03-19T20:15:00-03:00"
    ]
  },
  {
    "calendar": "h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com",
    "timeslot": [
      "2020-03-19T21:00:00-03:00",
      "2020-03-19T21:15:00-03:00"
    ]
  }
]


# Saving an event

In [70]:
request = service.events().insert(
    calendarId='h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com',
    body={
        "summary": 'Reiki do Roberto',
        "start": {
          "dateTime": "2020-03-19T21:00:00-03:00"
        },
        "end": {
            "dateTime": "2020-03-19T21:30:00-03:00"
        }
    }
)
# response = request.execute()

## Response example

```
{'kind': 'calendar#event',
 'etag': '"3169631813460000"',
 'id': 'ean1ava7ieopsdc6gt44i94tj8',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=ZWFuMWF2YTdpZW9wc2RjNmd0NDRpOTR0ajggaDU4dTFsOWNkMmY5aXMxOWdkZWM2dm00cXNAZw',
 'created': '2020-03-21T18:38:26.000Z',
 'updated': '2020-03-21T18:38:26.730Z',
 'summary': 'Reiki do Roberto',
 'creator': {'email': 'reikidaconceicao@gmail.com'},
 'organizer': {'email': 'h58u1l9cd2f9is19gdec6vm4qs@group.calendar.google.com',
  'displayName': '[RDC] Agenda do Washington',
  'self': True},
 'start': {'dateTime': '2020-03-19T21:00:00-03:00'},
 'end': {'dateTime': '2020-03-19T21:30:00-03:00'},
 'iCalUID': 'ean1ava7ieopsdc6gt44i94tj8@google.com',
 'sequence': 0,
 'reminders': {'useDefault': True}}
```